## API del Clima

Datos de Calidad de aire historico NuevaYorkCity https://open-meteo.com/en/docs/air-quality-api

#### Importamos librerias necesarias para nuestro trabajo

In [1]:
import pandas as pd  # Importar la librería pandas para manipulación y análisis de datos
import requests  # Importar el módulo requests para realizar solicitudes HTTP
import numpy as np

In [2]:
import requests
import pandas as pd

# Definimos la URL de la API con los parámetros deseados
url = 'https://air-quality-api.open-meteo.com/v1/air-quality?latitude=40.7143&longitude=-74.006&hourly=carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,aerosol_optical_depth&timezone=America%2FNew_York&start_date=2022-08-01&end_date=2024-03-15'

# Realizamos una solicitud GET a la API y almacenamos la respuesta
response = requests.get(url)

# Extraemos el contenido de la respuesta en formato JSON
json_data = response.json()

# Creamos un DataFrame de Pandas con los datos JSON
df_url = pd.DataFrame(json_data)

# Extraemos la columna 'hourly'
nuevo_df = pd.DataFrame(df_url['hourly'])

# Transponemos el DataFrame para tener las fechas y precipitaciones en columnas separadas
transpose_df = nuevo_df.transpose()

# Convertimos la columna 'time' y 'precipitation' en listas
nuevo_time = transpose_df['time'].explode('time')
carbon_monoxide = transpose_df['carbon_monoxide'].explode('carbon_monoxide')
nitrogen_dioxide = transpose_df['nitrogen_dioxide'].explode('nitrogen_dioxide')
sulphur_dioxide = transpose_df['sulphur_dioxide'].explode('sulphur_dioxide')
aerosol_optical_depth = transpose_df['aerosol_optical_depth'].explode('aerosol_optical_depth')

# Creamos un diccionario con las dos listas
diccionario = {'time': nuevo_time, 'carbon_monoxide_μg/m3': carbon_monoxide, 'nitrogen_dioxide_μg/m3': nitrogen_dioxide, 'sulphur_dioxide_μg/m3': sulphur_dioxide, 'aerosol_optical_depth_550nm': aerosol_optical_depth}

# Creamos un nuevo DataFrame a partir del diccionario
df_calidad_aire = pd.DataFrame(diccionario)

# Separamos la fecha y la hora de la columna 'time' en columnas separadas
fecha = df_calidad_aire['time'].str.split('T', expand=True)[0]
hora = df_calidad_aire['time'].str.split('T', expand=True)[1]

# Concatenamos las nuevas columnas con el DataFrame existente
df_calidad_aire_split = pd.concat([fecha, hora, df_calidad_aire], axis=1)

# Renombramos las columnas y reordenamos el DataFrame
df_calidad_aire_split = df_calidad_aire_split.rename(columns={0: 'fecha', 1: 'hora'})
df_calidad_aire_split = df_calidad_aire_split[['fecha', 'hora', 'carbon_monoxide_μg/m3', 'nitrogen_dioxide_μg/m3', 'sulphur_dioxide_μg/m3', 'aerosol_optical_depth_550nm']]

# Convertimos la columna 'fecha' a tipo datetime
df_calidad_aire_split['fecha'] = pd.to_datetime(df_calidad_aire_split['fecha'])

# Extraemos el año, mes y día de la columna 'fecha' y los agregamos como columnas separadas
df_calidad_aire_split['año'] = df_calidad_aire_split['fecha'].dt.strftime('%Y')
df_calidad_aire_split['mes'] = df_calidad_aire_split['fecha'].dt.strftime('%m')
df_calidad_aire_split['dia'] = df_calidad_aire_split['fecha'].dt.strftime('%d')

# Reordenamos las columnas
calidad_aire_2022_2024 = df_calidad_aire_split[['fecha', 'año', 'mes', 'dia', 'hora', 'carbon_monoxide_μg/m3', 'nitrogen_dioxide_μg/m3', 'sulphur_dioxide_μg/m3', 'aerosol_optical_depth_550nm']]

# Mostramos el DataFrame final
calidad_aire_2022_2024.dropna()

,fecha,año,mes,dia,hora,carbon_monoxide_μg/m3,nitrogen_dioxide_μg/m3,sulphur_dioxide_μg/m3,aerosol_optical_depth_550nm
68,2022-08-03,2022,08,03,20:00,316.0,80.3,11.1,0.09
69,2022-08-03,2022,08,03,21:00,303.0,71.8,10.4,0.1
70,2022-08-03,2022,08,03,22:00,284.0,59.7,9.6,0.11
71,2022-08-03,2022,08,03,23:00,261.0,44.8,8.6,0.11
72,2022-08-04,2022,08,04,00:00,247.0,37.0,8.1,0.12
...,...,...,...,...,...,...,...,...,...
14227,2024-03-15,2024,03,15,19:00,256.0,28.6,3.6,0.11
14228,2024-03-15,2024,03,15,20:00,272.0,33.6,3.9,0.19
14229,2024-03-15,2024,03,15,21:00,262.0,29.7,3.8,0.19
14230,2024-03-15,2024,03,15,22:00,248.0,24.7,3.6,0.18


In [3]:
# Verificar los tipos de datos de las columnas
print(calidad_aire_2022_2024.dtypes)


fecha                          datetime64[ns]
año                                    object
mes                                    object
dia                                    object
hora                                   object
carbon_monoxide_μg/m3                  object
nitrogen_dioxide_μg/m3                 object
sulphur_dioxide_μg/m3                  object
aerosol_optical_depth_550nm            object
dtype: object


In [4]:
# Convertir la columna a numérica
calidad_aire_2022_2024['año'] = pd.to_numeric(calidad_aire_2022_2024['año'], errors='coerce')
calidad_aire_2022_2024['mes'] = pd.to_numeric(calidad_aire_2022_2024['mes'], errors='coerce')
calidad_aire_2022_2024['carbon_monoxide_μg/m3'] = pd.to_numeric(calidad_aire_2022_2024['carbon_monoxide_μg/m3'], errors='coerce')
calidad_aire_2022_2024['nitrogen_dioxide_μg/m3'] = pd.to_numeric(calidad_aire_2022_2024['nitrogen_dioxide_μg/m3'], errors='coerce')
calidad_aire_2022_2024['sulphur_dioxide_μg/m3'] = pd.to_numeric(calidad_aire_2022_2024['sulphur_dioxide_μg/m3'], errors='coerce')
calidad_aire_2022_2024['aerosol_optical_depth_550nm'] = pd.to_numeric(calidad_aire_2022_2024['aerosol_optical_depth_550nm'], errors='coerce')
calidad_aire_2022_2024['dia'] = pd.to_numeric(calidad_aire_2022_2024['dia'], errors='coerce')

In [5]:
calidad_aire_ugm3_2023_2024 = calidad_aire_2022_2024[calidad_aire_2022_2024['año'] >= 2023]
calidad_aire_ugm3_2023_2024

,fecha,año,mes,dia,hora,carbon_monoxide_μg/m3,nitrogen_dioxide_μg/m3,sulphur_dioxide_μg/m3,aerosol_optical_depth_550nm
3672,2023-01-01,2023,1,1,00:00,549.0,54.5,3.9,0.06
3673,2023-01-01,2023,1,1,01:00,461.0,52.4,3.2,0.06
3674,2023-01-01,2023,1,1,02:00,356.0,46.9,2.5,0.06
3675,2023-01-01,2023,1,1,03:00,301.0,38.0,2.2,0.05
3676,2023-01-01,2023,1,1,04:00,256.0,26.5,2.0,0.05
...,...,...,...,...,...,...,...,...,...
14227,2024-03-15,2024,3,15,19:00,256.0,28.6,3.6,0.11
14228,2024-03-15,2024,3,15,20:00,272.0,33.6,3.9,0.19
14229,2024-03-15,2024,3,15,21:00,262.0,29.7,3.8,0.19
14230,2024-03-15,2024,3,15,22:00,248.0,24.7,3.6,0.18


Guardaos nuestra data de calidad de aire 2023 al 2024

In [6]:
calidad_aire_ugm3_2023_2024.to_parquet("../0-DataSets/3-DatosScrapingConETL/calidad_aire_ugm3_2023_2024.parquet")